# 순환 신경망 고급으로 사용하기
> 순환 싱경망의 성능과 일반화 능력의 향상을 위한 기술

* __순환 드롭아웃__ : 순환 층에서 과대적합 방지를 위해 케라스에 내장되어 있는 드롭아웃 사용
* __스태킹 순환 층__ : 네트워크의 표현 능력 증가
* __양방향 순환 층__ : 순환 네트워크에 같은 정보를 다른 방향으로 주입하여 정확도와 기억력 향상

### 데이터 준비 -> 24시간 후의 온도 예측
> lookback 타임스텝만큼 돌아가서 매 steps 타임스텝마다 샘플링  

#### __사용할 변수__
* lookback = 1440: 10일 전 데이터로 돌아간다
* steps = 6: 1시간마다 데이터 포인트 하나를 샘플링
* delay = 144: 24시간이 지난 데이터가 타깃이 된다.

#### __처리해야 하는 작업__
* 데이터 전처리 -> 정규화
* float_data 배열을 받아 데이터 배치와 미래 타깃 온도를 추출하는 파이썬 제너레이터를 만듦

In [22]:
import os
import numpy as np

fname = os.path.join('./datasets/jena_climate', 'jena_climate_2009_2016.csv')
f = open(fname)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

# 데이터 파싱
float_data = np.zeros((len(lines), len(header) -1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values

# 데이터 정규화
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std

#### 제너레이터 함수는 (입력 데이터로 사용할 배치, 타깃 온도의 배열) 튜플을 반복적으로 반환한다.
* data : 정규화한 부동 소수 데이터로 이루어진 원본 배열
* lookback : 입력으로 사용하기 위해 거슬러 올라갈 타임스텝
* delay : 타깃으로 사용할 미래의 타임스텝
* min_index와 max_index : 추출할 타임스텝의 범위를 지정하기 위한 data 배열의 인덱스. 검증 데이터와 테스트 데이터를 분리하는 데 사용
* shuffle : 샘플을 섞을지, 시간 순서대로 추출할지 결정
* batch_size : 배치의 샘플 수
* step : 데이터를 샘플링할 타임스텝 간격. 1시간에 하나의 데이터 포인트를 추출하기 위해 6으로 지정

In [25]:
# 시계열 데이터와 타깃을 반환하는 제너레이터 함수
def generator(data, lookback, delay, min_index, max_index, shuffle=False, batch_size=128, step=6) :
    if max_index is None :
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rwos = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rwos[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets

# 훈련용(20만개의 타임스텝), 검증용(10만개의 타임스텝), 테스트용(나머지) 제너레이터 제작
lookback = 1440
step = 6
delay=144
batch_size = 128
train_gen = generator(float_data,
                     lookback=lookback,
                     delay=delay,
                     min_index=0,
                     max_index=200000,
                     shuffle=True,
                     step=step,
                     batch_size=batch_size)
val_gen = generator(float_data,
                   lookback = lookback,
                   delay=delay,
                   min_index=200001,
                   max_index = 300000,
                   step=step,
                   batch_size=batch_size)
test_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=300001,
                    max_index=None,
                    step=step,
                    batch_size=batch_size)

val_steps = (300000 - 200001 - lookback)  // batch_size  # 전체 검증 세트를 훈회하기 위해 val_gen에서 추출할 회수

test_steps = (len(float_data) - 300001 - lookback) // batch_size  # 전체 테스트 세트를 순회하기 위해 test_gen에서 추출할 회수

### 첫번째 신경망
> LSTM 보다 간단한 GRU로 구현

In [26]:
from keras.models import Sequential
from keras import layers
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(layers.GRU(32, input_shape=(None, float_data.shape[-1])))
model.add(layers.Dense(1))

model.compile(optimizer='RMSprop', loss='mae')
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=40,
                              validation_data=val_gen,
                              validation_steps=val_steps)

C:\Users\김보석\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


UnboundLocalError: local variable 'rows' referenced before assignment